### Running PESTPP-IES

In [ ]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd

In [ ]:
# specify the temporary working folder
t_d = os.path.join('pst_template_ies')
# get the previously generated PEST dataset
org_t_d = os.path.join("pst_template_pmc")
if not os.path.exists(org_t_d):
    raise Exception()
if os.path.exists(t_d):
    shutil.rmtree(t_d)
shutil.copytree(org_t_d,t_d)

In [ ]:
pst_path = os.path.join(t_d, 'pest.pst')
pst = pyemu.Pst(pst_path)

In [ ]:
# check to see if obs&weights notebook has been run
if not pst.nnz_obs > 0:
    raise Exception()

In [ ]:
pst.pestpp_options["ies_num_reals"] = 100
pst.pestpp_options["ies_bad_phi_sigma"] = 1.75
pst.pestpp_options["ies_multimodal_alpha"] = 0.99

In [ ]:
obs = pst.observation_data
obs["standard_deviation"] = np.nan
nzobs = obs.loc[obs.weight>0,:].copy()
obs.loc[nzobs.obsnme,"standard_deviation"] = 0.33
dobs = nzobs.loc[nzobs.obsnme.str.contains("gde"),:]
assert dobs.shape[0] == 1
obs.loc[dobs.obsnme,"standard_deviation"] = dobs.obsval * 0.2
nzobs = obs.loc[obs.weight>0,:].copy()
np.random.seed(pyemu.en.SEED)
draws = np.random.normal(0.0,1.0,pst.pestpp_options["ies_num_reals"])
vals = nzobs.obsval.values
stdevs = nzobs.standard_deviation.values
noisyobs = []
for i,d in enumerate(draws):
    noise = d * stdevs
    noisyobs.append(vals+noise)
df = pd.DataFrame(noisyobs,index=np.arange(pst.pestpp_options["ies_num_reals"]),columns=nzobs.obsnme)
df.to_csv(os.path.join(t_d,"corrnoise.csv"))
pst.pestpp_options["ies_obs_en"] = "corrnoise.csv"
    

In [ ]:
pst.control_data.noptmax = -2
pst.write(os.path.join(t_d, 'pest.pst'),version=2)

pyemu.os_utils.run("pestpp-ies pest.pst",cwd=t_d)

In [ ]:
with open(os.path.join(t_d,"phi.csv"),'w') as f:
    f.write("hds,0.5\n")
    f.write("drn,0.5\n")
pst.pestpp_options["ies_phi_factor_file"] = "phi.csv"

In [ ]:
pst.control_data.noptmax = -2
pst.write(os.path.join(t_d, 'pest.pst'),version=2)

pyemu.os_utils.run("pestpp-ies pest.pst",cwd=t_d)

In [ ]:
pst.control_data.noptmax = 3
pst.write(os.path.join(t_d, 'pest.pst'),version=2)

In [ ]:
num_workers = 20
m_d = "master_ies0"

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"pest.pst"))
obs = pst.observation_data
oe_pr = pst.ies.obsen0
oe_pt = pst.ies.get("obsen",pst.ies.phiactual.iteration.max())
forecasts = pst.pestpp_options["forecasts"].split(",")
for f in forecasts:
    ax = oe_pr.loc[:,f].plot(kind="hist",fc="0.5",alpha=0.5,density=True)
    ax = oe_pt.loc[:,f].plot(kind="hist",fc="b",alpha=0.5,density=True)
    ylim = ax.get_ylim()
    v = obs.loc[f,"obsval"]
    ax.plot([v,v],ylim,"r-",lw=2)
    ax.set_title(f)
    plt.show()
        

In [ ]:
hkobs = obs.loc[obs.oname=="hk",:].copy()
hkobs["i"] = hkobs.i.astype(int)
hkobs["j"] = hkobs.j.astype(int)


In [ ]:
fig,axes = plt.subplots(1,2,figsize=(10,5))
prarr = np.zeros((hkobs.i.max()+1,hkobs.j.max()+1))
prarr[hkobs.i,hkobs.j] = np.log10(oe_pr.loc["base",hkobs.obsnme])
ptarr = np.zeros((hkobs.i.max()+1,hkobs.j.max()+1))
ptarr[hkobs.i,hkobs.j] = np.log10(oe_pt.loc["base",hkobs.obsnme])
vmin = min(prarr.min(),ptarr.min())
vmax = max(prarr.max(),ptarr.max())
print(ptarr.max())
print(prarr.min())
print(vmin,vmax)
cb = axes[0].imshow(prarr,vmin=vmin,vmax=vmax)
plt.colorbar(cb,ax=axes[0])
cb = axes[1].imshow(ptarr,vmin=vmin,vmax=vmax)
plt.colorbar(cb,ax=axes[1])

In [ ]:
fig,ax = plt.subplots(1,1)
noise = pst.ies.noise
hobs = obs.loc[(~obs.obsnme.str.contains("gde")) & (obs.weight>0),:]
hobs.sort_values(by="obsval",inplace=True)
hvals = hobs.obsval.values
for real in oe_pt.index:
    vals = noise.loc[real,hobs.obsnme].values
    ax.plot(hvals,vals,"r-",marker=".",alpha=0.5)
    vals = oe_pt.loc[real,hobs.obsnme].values
    ax.plot(hvals,vals,"b",marker=".",alpha=0.5)
xlim = ax.get_xlim()
ylim = ax.get_ylim()
mn = min(xlim[0],ylim[0])
mx = max(xlim[1],ylim[1])
ax.plot([mn,mx],[mn,mx],"k--",lw=3)
ax.set_xlim(mn,mx)
ax.set_ylim(mn,mx)


    